<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Linear Opamp 04   
# Non Inverting Amplifier

<BR>

This project deals with one of the most common opamp topologies: **the non inverting amplifier**. 
Using this circuit the basics of linear opamp circuits bandwidth will be explained.

Version 1.1 (13/3/2019)  
License information is at the end of the document

---

**Bill Of Materials (BOM):**

* 1x Dual Opamp MCP6002
* Resistors: 2x $1 k\Omega$, 2x $10 k\Omega$, 1x  $22 k\Omega$ and 2x $100 k\Omega$
* Capacitors: 1x  $100 nF$

---

## The non inverting topology

The following circuit is a non inverting amplifier that generates an output voltage that is greater that the input. In the extreme case of choosing R2 infinite, the circuit defaults to our beloved follower.

![fig01](images\L_OA_04\fig01.png)

We can analyze this circuit using the ideal zero order model. If you assume that it does not saturate, the **virtual short circuit** assumption $V_{(+)} \approx V_{(-)}$ can be used.

In order to measure the circuit we will choose $R1 = 22 k\Omega$ and $R2 = 10k\Omega$.

---

**THEORETICAL TASK**

Determine the Vo(Vi) relationship as function of R1 and R2 for the above circuit.  
Assume virtual short circuit.
Obtain the relationship for the selected resistor values.

---


## Measuring the circuit

Now that we have calculated the circuit, let's measure it.

---

![Practical Icon](images/pt.png)

---

Mount the proposed circuit.  
Remember to add the ADC and DAC connections.  

We will first import the **SLab** module and **connect** with the board.

In [ ]:
# Import modules
import slab
import slab.dc as dc

In [ ]:
boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

It is always a good idea to check the board calibration.   
The following code cell checks the calibration of the **DACs** and the first four **ADCs**

In [ ]:
# Check the calibration
slab.checkCalibration(pause=False,na=4)

Now we will obtain a a Vo(Vi) curve between 0V an 3.2V in 0.1V steps using the **curveVV** function of the **slab.dc** module.

Observe that we use the option **_adc2=True_**. That makes SLab use the input voltage measurements from ADC2 instead of reliying on the DAC programmed values. This is important when measuring too close to Ground or Vdd.

In [ ]:
# Amplifier DC curve
dc.curveVV(0,3.2,0.1,adc2=True)

We can ease the gain calculation. From the figure we know that we are out of the saturation region for voltages below 1V. We can obtain the Vo value for two known Vi values to calculate the gain. To get better measurements we won't rely on the programmed DAC values and we will measure both Vi and Vo using the ADCs.

In [ ]:
# Measure output at 0.7V input
slab.setVoltage(1,0.7)
voa = slab.readVoltage(1)
via = slab.readVoltage(2)

# Measure output at 0.3V input
slab.setVoltage(1,0.3)
vob = slab.readVoltage(1)
vib = slab.readVoltage(2)

# Calculate the gain
Gain = (voa - vob) / (via - vib)
print ('Gain =',Gain)

Execute the previous cell to measure the DC gain of the circuit.  
Do you obtain the expected value?  

Before continuing it is good to examine the lower region of the curve.

Execute the following code to show the circuit response for voltages lower than 0.2V

In [ ]:
# Curve close to zero
dc.curveVV(0,0.2,0.01,adc2=True)

Unless you have a very good driver circuit, the line probably does not pass through the (0,0) point.

The DACs and the Operational amplifiers don't work too well **very close to ground**.

We can also test the circuit against a time varying signal. The SLab system can generate different waveforms. We will first select a sine waveform. Our amplifier features a gain below 4, so a signal less than 800 mV in the high peak is adequate. We will select a high peak of 800 mV and a low peak of 400 mV.

Waveforms are generated sending a set of samples. In our case we want a pretty detailed sine so we will use 100 samples for one cycle. 

>`slab.waveSine(0.4,0.8,100)`

The **SLab** system will respond with something like:

>`100 point wave loaded`  
>`Wave frequency must be between 0.000100 and 400.00 Hz`  
>`Current frequency is 10.0 Hz`

The maximum sample rate for the hardware board depend on its firmware. For instance, if the maximum sample rate at which the DAC can generate samples is 40 kHz, then, if you use 100 samples for one waveform, the maximum frequency of this waveform is 400 Hz.

When the system starts the default sample rate is 1 kHz so, if you send the 100 samples at a sample frequency of 1 kHz, you get a 10 Hz sinewave.

We don't want a 10 Hz frequency so we will change it to 100 Hz:

>`slab.setWaveFrequency(100)`

The last step before performing the measurement is selecting how much data we want to obtain. As 100 samples get one wave, we will select 500 samples to get 5 full waves. We also want to obtain data from 2 ADCs (ADC 1 and ADC 2).

>`slab.tranStore(500,2)`

Now we can finally obtain the wave response. The wavePlot command generates the wave during the time indicated in the previous setTransientStorage command. You can see the command details in the SLab Python reference document.

>`slab.wavePlot()`

The following code executes the above commands.

In [ ]:
# Sine wave response
slab.waveSine(0.4,0.8,100)
slab.setWaveFrequency(100)
slab.tranStore(500,2)
slab.wavePlot()

That should give you a nice presentation of the input signal Vi (ADC 2) and the output signal VO (ADC 1). Both signals should be in phase as you are well below the band pass limit of the circuit.

You can measure both signals amplitudes to compute the gain, but there is an easier way by using the analize command.
We can instruct the SLab module to return the data of the wavePlot by setting its optional parameter returnData to True. This way, you will get a list returned after the plot.

>`data = slab.wavePlot(returnData=True)`

The list will contain in the 0 position a vector with the independent X axis data. From positions 1 onwards you will get all Y axis curves. In our case the wavePlot command draws two curves so we will get a list of three elements: DAC 1, ADC 1 and ADC 2. 

Now we can analyze the data contents using the analyze command. This command is part of the meas module, so we will need to import it.
This command will give a lot of information, including peak to peak values of information contained in the data list.

>`import slab.meas as meas`  
>`meas.analyze(data)`

From the **peak to peak** information of each signal it is easy to compute the gain.

Note that, as the amplifier also amplifies the DC voltage, you can also obtain the gain from the **mean value** of the signals.

In [ ]:
# Import the measurement module
import slab.meas as meas

# Wave measurement returning measurement data
data = slab.wavePlot(returnData=True)

# Measurement analysis
meas.analyze(data)

After perfoming the proposed measurement you obtain the gain from the peak to peak values.   
Do you obtain the same result as in the DC case?   

It could be interesting to measure the bandwidth of this circuit. Unfortunately it is too high to be directly measured for the current SLab system.

## Amplifier Bandwidth

<div class="alert alert-block alert-warning"><font color=black>
We now will use circuit analysis in the "s" domain. It is recommended to have knowledge on this subject to continue. 
</font></div>

The model of the amplifier, based on **virtual shortcircuit**, predicts an amplification that is independent on the frequency of the input signal. It is not reasonable to expect the amplifier to work the same with any signal at any frequency.
To go beyond the virtual shortcircuit model we need to use a better opamp model We will use the first order model for the opamp with a DC gain $A_O$ and a dominant pole $p1$.

$$V_O = A(s)\left(V_{(+)}-V_{(-)}\right) \qquad A(s) = \frac{A_O}{1+\frac{s}{p1}}$$

We also have the topology of the circuit, so we can start doing serious work.

---

**THEORETICAL TASK**
Obtain the frequency response H1(s) = Vo/Vi for the previous circuit.  
Use the first order opamp parameters p1 and Ao and the resistor values R1, R2 as unknowns.  
As usual, you can assume Ao much greater than 1.   

---

There should be only one pole and it should be in the negative region of the "s" plane.

As we will implement the circuit using the MCP6002 opamp, we can perform some calculations specific to this device. Remember the data for this opamp:

$$A_O = 112 dB \qquad GBW = 1MHz$$

For now we will also select the resistor values in our current circuit:

$\qquad R1 = 2.2 k\Omega$ &nbsp; and &nbsp; $R2 = 1 k\Omega$

---

**THEORETICAL TASK**
Give values to the DC  gain and total bandwidth of the circuit for considered case of R1 and R2    
Check that the DC gain is 3,2 and the bandwidth is about 312 kHz    

---
    
Of course, the $H_1(0)$ value has been previously calculated in a much easier way by applying virtual short circuit on the (+) and (-) inputs of the opamp. From that, if the GBW product is conserved, then we can just apply:

$$BW = \frac{GBW}{H(0)}$$

The fast to calculate virtual short circuit method gives the same results but they are not guaranteed. We needed to perform the real calculations so we can be sure that:

1. The final closed loop system pole is in the negative "s" plane.

2. The GBW product really is conserved.

The virtual shortcircuit method can give the good result but it can also give a wrong result, try to change **V(+)** for **V(-)** if you are not convinced. As you see the problem is that this method does not give any way to check if we are getting the good result.

It could be interesting to measure the circuit to obtain the bandwidth. Unfortunately, the SLab system is quite limited in its frequency measurements. We need to increase the gain so that the bandwidth is low enough to be measured.

## Increasing the gain

Let’s move on to bigger gains. 
Change **R2** to $1 k\Omega$ and **R1** to a $100 k\Omega$ resistor. 

As the gain is very high, a very small input voltage can saturate the amplifier. In fact, it takes less than 50mV input voltage to saturate output.

The following code obtains the Vo(Vi) curve for input voltages lower than 0.1V

In [ ]:
# Low voltage DC response
dc.curveVV(0,0.1,0.005,adc2=True)

Depending of the buffer circuits in the hardware board, the obtained curve can be from quite good to very bad.

Althoug the **buffers** we use in the SLab system are full rail, they don't operate too well too close to ground as we should have seen in previous experiments. One solution to provide proper measurements is using a virtual ground.

![fig02](images\L_OA_04\fig02.png)

In the shown circuit we have added the DAC 2 voltage to the terminal of R2 where was ground before. Now, the **virtual ground** voltage of the circuit is the voltage set on DAC 2. If we set this DAC 2 to 1V, 1V will be our new ground, so all voltages shall be now refered to the new ground:

$$\qquad V'_x = V_x - 1V$$

Instead of substracting 1V to the measurements we will subtracte the measurement on ADC 3 to all input (ADC 2) and output (ADC1) measurements.

As the gain will be higher, in order to obtain the DC Vo(Vi) curve we need to use a DC sweep step low enough to have more than two points in the linear region. We will use a 5mV step. Check that it is adequate. It makes no sense to sweep all the input range from 0 V to 3.3 V as at such high gains the output will be saturated for most of the range.

---

**BUILD TASK**
Mount the proposed circuit.   
Don't forget to add the ADC 3 connection.    

---

We can obtain the curve for a 100 mV arround the set DAC 2 value. As the voltage range is small, we will increase the number of readings SLab average at each point to reduce the measurement noise.

In [ ]:
# Set the virtual ground voltage at DAC 2 to 1V
slab.setVoltage(2,1)

# Do 200 readings at each point
old = slab.setDCreadings(200)

# Perform a DC sweep
dac,adc1,adc2,adc3,*rest = slab.dcSweep(1,0.95,1.05,0.005)
Vi = adc2 - adc3
Vo = adc1 - adc3
slab.plot11(Vi,Vo,'DC Plot','Vi (V)','Vo (V)')

Perhaps the curve is not perfect but it should pass quite close to the (0,0) point.

We can use two non saturated points to obtain the gain.

In [ ]:
# Virtual ground voltage
slab.setVoltage(2,1)

# Measurement at virtual ground
slab.setVoltage(1,1)
voa = slab.readVoltage(1)
via = slab.readVoltage(2)

# Measurement above virtual ground
slab.setVoltage(1,1.01)
vob = slab.readVoltage(1)
vib = slab.readVoltage(2)

# Gain calculation
Gain = (voa - vob) / (via - vib)
print ('Gain =',Gain)

You should obtain a gain quite close to the theoretical value. Check it.

If it is not correct, check that the two measurement points are in the non saturated region of the curve.

Remember that the resistances have tolerance, so, the obtained gain should be inside the tolerance region:

$$Gain = 1 + \frac{R1 (1 \pm tol)}{R2 (1 \pm tol)}$$

The following code calculates the gain limits due to the tolerance values:

In [ ]:
# Values
R1 = 100000 # Ohm
R2 =   1000 # Ohm
tol = 0.05

Gain_max = 1 + (R1*(1+tol))/(R2*(1-tol))
Gain_min = 1 + (R1*(1-tol))/(R2*(1+tol))

print('Maximum gain:',Gain_max)
print('Minimum gain:',Gain_min)

Any value inside the above range can be attributed a resistor tolerance errors. Note that the above cases are worst case limits with very low probability of occurrence.

If you get a value very close to the theoretical value, this is probably because the resistors are not near their tolerance limits.

## Check against a sine wave

Now we want to check our high gain amplifier against a sine wave. The problem is that we need to provide a very low amplitude wave so that the amplifier does not saturate. To obtain a 2V output range with a 100 gain amplifier we need a 20mV input range. This is too close to the quantization steps of the DACs.

A way to imporve the signal quality is to use an attenuator previous to the amplifier as shown in the next image.

![fig03](images\L_OA_04\fig03.png)

The attenuator will reduce the input voltage. For voltages respect to the **real ground** we get:

$$V_i = V_{DAC 2} + (V_{DAC 1} - V_{DAC 2})\frac{R4}{R3+R4}$$

For voltages respect to the **virtual ground** set by DAC 2, the expression is much simpler:

$$V_i = V_{DAC 1} \frac{R4}{R3+R4}$$

If we add the amplifier gain we get:

$$V_o = V_{DAC 1} \frac{R4}{R3+R4} Gain$$

So the amplifier gain can be calculated:

$$Gain = \frac{V_o}{V_{DAC 1}} \frac{R3+R4}{R4}$$

In our case we will select $R3 = 100k\Omega$ and $R4 = 1k\Omega$

---

**BUILD TASK**  
Mount the proposed circuit that includes the attenuator.    

---

Now we can calculate the attenuation factor due to R3 and R4:

$$factor = \frac{R3+R4}{R4}$$

And measure the response of the circuit:

In [ ]:
# Gain factor
R3 = 100000 # Ohm
R4 = 1000   # Ohm
factor = (R3+R4)/R4
print('Attenuation factor:',factor)
print()

# Sine wave generation
slab.waveSine(1,2,100)
slab.setWaveFrequency(100)
slab.tranStore(500,2)
slab.wavePlot()

In the figure, input voltage on ADC2, should be almost equal to output voltage on ADC1, but, in this case, that means that we have an amplifier gain that is equal to the attenuation factor.

As we are using the same resistor values both on the attenuator and the amplifier, they compensate to each other.

## Bode Plot

You can compute the expected bandwidth of the using the previous calculations.

$$BW = \frac{GBW}{H(0)}$$

Now we want to generate a Bode plot that shows the frequency response of the circuit. As we have included an attenuator, the plot will also include the response of the attenuator. This attenuator is a resistive network so it has no poles or zeros. We should expect to obtain a DC gain of 1 (0 dB) with the attenuator included. 

We can obtain the Bode plot using the bodeResponse command. This command is part of the **slab.ac** module, so we will need to import it.  
We need to provide as parameters:

* Low peak of the input signal (1 V)
* High peak of the input signal (2V)
* Minimum frequency (10 Hz)
* Maximum frequency (8 kHz)

We also set to true the **returnData** parameter to obtain, in the **data** variable the measurement results.

In [ ]:
# Import the SLab AC module
import slab.ac as ac

# Draw a bode plot
data = ac.bodeResponse(1,2,10,15000,returnData=True)

The above graphs show the bode plot from the input, set on **DAC1** to the output, read on **ADC1**. That includes both the amplifier and the attenuator.

What we want is the bode plot of the amplifier only, so we need to take out the effect of the attenuator.

Execute the following **code** to show the bode curves of the amplifier:

In [ ]:
# Take the frequency vector
f = data[0]       # Frequency vector

# Take the response vector and compensate for the attenuation
g = 101*data[1]   # Compensate attenuation

# Draw the bode plot
ac.plotBode(f,g)  # Draw the bode plot

A gain of $40dB$ corresponds to a linear gain of 100. You should be able to obtain the pole position from the above graph.

Check the Bode plot and calculate both the gain and the bandwidth.    
Has the bandwidth the expected value.   
Is the GBW product conserved?   

## DC Pass-through

Now that we have analyzed the basic non inverter amplifier, let's see one of its useful variations:

![fig03](images\L_OA_04\fig04.png)

In this circuit we have a capacitor $C_2$ in series with resistor $R_2$. 

What does $C_2$ in our circuit? In short, it adds a pole and a zero. As the original amplifier featured one pole due to the GBW product, we end up with one zero and two poles. As the two poles are not related, all values are real. Nothing imaginary here.

**Three ways to skin a cat**

There are several ways to calculate the $H2(s)$ transfer function of this circuit.

**(1) First option is the brute force attack.** Substitute the first order opamp model on the circuit and solve all the equations until you find the final solution: 

$$H_2(s)=\frac{V_o}{V_i}$$

This is a long way but it can be done.

**(2) Second option is the feedback theory method.** Separate the system in two transfer functions:

$$a(s) = \frac{Vo}{Vd}(s) \qquad f(s) = \frac{V_{(-)}}{V_o}(s)$$

Where $a(s)$ is the first order opamp gain that depends on **Ao** and **p1**, and $f(s)$ is the feedback function that depends on $R_1$, $R_2$ and $C_1$ that relates the output voltage $Vo$ with the $V_{(-)}$ input. Once we know $a(s)$ and $f(s)$, then we can do some calculations:

$$V_o = a(s)V_d = a(s) \left(V_{(+)}-V_{(-)}\right)
= a(s)\left(V_i-f(s)V_o \right)$$

That gives:

$$H_2(s) = \frac{V_o}{V_i}=\frac{a(s)}{1+a(s)\cdot f(s)}$$

Substitute the known values of $a(s)$ and $f(s)$, operate and you are done.

**(3) Third option is the divide and conquer lazy method.**

We start using the virtual short circuit method that provides the low frequency response of the circuit $H_{LF}(s)$. As this model doesn’t include the opamp pole, it cannot predict the bandwidth of the circuit. That’s fine for now.

From virtual short circuit, use the following equations to obtain $H_{LF}(s)$ :

$$V_{(+)} = V_{(-)} \qquad i = \frac{V_o-V_i}{R_1}
\qquad V_o = i \left(R_1 + R_2 + \frac{1}{C_2\cdot s} \right)
\qquad H_{LF}(s) = \frac{V_o}{V_i}$$

The $H_{LF}(s)$ features a zero and a pole. 
As explained, due to the use of virtual short circuit, the operational pole **p1** is not considered. The expected behavior of $H_{LF}(s)$, as the zero is smaller than the pole, should be like that:

![Zero and Pole](images\L_OA_04\zero_pole.png)

Where **AM** is the medium frequency gain that should be the same as the $H_1(0)$ value we calculated previously. This makes sense: $C_2$ is the component that produces $Z_L$ and $P_L$. At frequencies much higher than $P_L$, $C_2$ behaves like a short circuit, so the operation of the circuit is the same than the one of the original non inverting amplifier.

We now calculate the high frequency behavior of the circuit. We know that we are limited by the opamp GBW, that way, the high frequency pole shall be:

$$P_H = \frac{GBW}{A_M}$$

If $P_L$ is much lower than the calculated $P$H$, then we know that the GBW doesn’t affect the low frequency behavior of the circuit and we can just add the new pole to the system:

![Full Response](images\L_OA_04\full_response.png)


---

**THEORETICAL TASK**  
Solve the circuit using, at least one of the three proposed methods.   
Obtain the zero value ZL and the two pole values PL and PH.   
Obtain also the DC gain H2(0)     

If you want to learn more, feel free to use all three and compare the results. 

---

The behavior of the system is as follows:

* At low frequencies, much below $Z_L$, capacitor behaves as an open circuit, so the circuit behaves like a follower.

* At medium, much higher than $P_L$ and much lower than $P_H$, the capacitor behaves as a short circuit so the circuit behaves as a normal inverter in DC operation.

* At high frequencies the GBW product kicks in as in the original amplifier.

We will select the low gain resistor values $R1 = 22 k\Omega$ and $R2 = 10k\Omega$. For $C_2$ we will choose a $100nF$ value. 

---

**THEORETICAL TASK**  
Obtain H2(0) and the zero and poles frequencies for the selected components.   

---

Now it is time to test the circuit.

---

**BUILD TASK**  
Mount the proposed circuit.   

---

# Import the SLab AC module

Remember the command to get the bode plot. This time we don't aspire to reach the second pole as its frequency is too high for the SLab system, so we will limit ourselves to a frequency of 5 kHz. We will set the peak to peak amplitude to $0.2V$. As the circuit blocks the DC component, we can set the DC value of the input signal wherever we want.

In [ ]:
# Obtain the bode response
ac.bodeResponse(1.4,1.6,5,5000)

We can check that the gain is one (0 dB) at DC by obtaining a $V_o(V_i)$ curve.

In [ ]:
# DC response
dc.curveVV(0,3.2,0.1)

As the capacitor behaves as an open circuit at DC, the circuit becomes a follower.

You can do the same by using a sine wave if it is below the first zero. Gain should be one. We use a peak to peak amplitude of $1V$ in this case.

In [ ]:
# Response to a 5Hz sine wave
slab.waveSine(1,2,100)
slab.setWaveFrequency(5)
slab.tranStore(500,2)
slab.wavePlot()

Over the first pole you get the gain defined by the resistor ratio. We use a peak to peak amplitude of $0.2 V$ to prevent the amplifier saturation. We also reduce the number of samples per wave because the maximum sample rate of the SLab system could not allow us to have 100 samples per wave for a wave frequency of 1 kHz.

In [ ]:
# Response to a 1000Hz sine wave
slab.waveSine(1.4,1.6,20)
slab.setWaveFrequency(1000)
slab.tranStore(100,2)
slab.wavePlot()

Observe, also, that in this circuit, the DC is conserved, so the input and output signals have the same average value. Last measurement featured a $1.5 V$ DC value, now we test with a $2.5 V$ DC value.

In [ ]:
# Response to a 1000Hz sine wave with 2.5V offset
slab.waveSine(2.4,2.6,20)
slab.wavePlot()

## Last comments

In this project we have seen different circuits based on the non inverting circuit topology.
We also have seen the conservation of the GBW product. 
Keep note of that result because it will be an important point in the opamp inverting circuit.

## References

[SLab Python References](../Reference)  
Those are the reference documents for the SLab Python modules. They describe the commands that can be carried out after importing each module. 
They should be available in the SLab/Doc folder.

[TinyCad](https://sourceforge.net/projects/tinycad/)  
Circuit images on this document have been drawn using the free software TinyCad  

[SciPy](https://www.scipy.org/)  
All the functions plots have been generated using the Matplotlib SciPy package.  

## Document license

Copyright  ©  Vicente Jiménez (2018-2019)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">